In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\MLOps\\CICD\\End-to-End-ML'

In [6]:
import pandas as pd

In [7]:
df=pd.read_csv("artifacts/data_ingestion/bankchurn.csv")
df.head()

,Unnamed: 0,RowNumber,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0,1,15634602,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,1,2,15647311,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,2,3,15619304,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,3,4,15701354,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,4,5,15737888,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         10000 non-null  int64  
 1   RowNumber          10000 non-null  int64  
 2   CustomerId         10000 non-null  int64  
 3   CreditScore        10000 non-null  int64  
 4   Gender             10000 non-null  int64  
 5   Age                10000 non-null  int64  
 6   Tenure             10000 non-null  int64  
 7   Balance            10000 non-null  float64
 8   NumOfProducts      10000 non-null  int64  
 9   HasCrCard          10000 non-null  int64  
 10  IsActiveMember     10000 non-null  int64  
 11  EstimatedSalary    10000 non-null  float64
 12  Exited             10000 non-null  int64  
 13  Geography_Germany  10000 non-null  int64  
 14  Geography_Spain    10000 non-null  int64  
dtypes: float64(2), int64(13)
memory usage: 1.1 MB


In [10]:
df.columns

Index(['Unnamed: 0', 'RowNumber', 'CustomerId', 'CreditScore', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [11]:
from dataclasses import dataclass
from pathlib import Path

In [24]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE:str
    unzip_data_dir:Path
    all_schema:dict



In [29]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
       self, 
       config_filepath=CONFIG_FILE_PATH,
       params_filepath=PARAMS_FILE_PATH,
       schema_filepath=SCHEMA_FILE_PATH):     
       
       self.config=read_yaml(config_filepath)
    #    self.params=read_yaml(params_filepath)
       self.schema=read_yaml(schema_filepath)
       create_directories([self.config.artifacts_root])

    def get_data_validation_config(self)->DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config





In [26]:
import os
from mlProject import logger

In [34]:
class DataValidation:
    def __init__(self, config:DataValidationConfig):
        self.config=config
    def validate_all_columns(self)->bool:
        try:
            validation_status=None
            data=pd.read_csv(self.config.unzip_data_dir)
            data.drop(['Unnamed: 0', 'RowNumber'], axis=1, inplace=True)
            all_cols=list(data.columns)

            all_schema=self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                    break

                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status} ")

            return validation_status
        except Exception as e:
            raise e

    



In [36]:
config=ConfigurationManager()
data_validation_config=config.get_data_validation_config()
data_validation=DataValidation(config=data_validation_config)
data_validation.validate_all_columns()

[2024-03-10 13:48:18,164:INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-03-10 13:48:18,172:INFO:common: yaml file: schema.yaml loaded successfully]
[2024-03-10 13:48:18,172:INFO:common: created directory at: artifacts]
[2024-03-10 13:48:18,180:INFO:common: created directory at: artifacts/data_validation]


True

In [17]:
df.columns

Index(['Unnamed: 0', 'RowNumber', 'CustomerId', 'CreditScore', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

,RowNumber,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,1,15634602,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,2,15647311,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,3,15619304,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,4,15701354,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,5,15737888,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,771,1,39,5,0.00,2,1,0,96270.64,0,0,0
9996,9997,15569892,516,1,35,10,57369.61,1,1,1,101699.77,0,0,0
9997,9998,15584532,709,0,36,7,0.00,1,0,1,42085.58,1,0,0
9998,9999,15682355,772,1,42,3,75075.31,2,1,0,92888.52,1,1,0
